In [ ]:
from google.colab import drive
drive.mount('/content/drive')
!pip install py7zr

Mounted at /content/drive
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.4/66.4 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 53.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 399.3/399.3 kB 42.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.8/138.8 kB 19.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.8/49.8 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 74.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.1/93.1 kB 11.7 MB/s eta 0:00:00


In [ ]:
import py7zr

with py7zr.SevenZipFile('/content/drive/MyDrive/data.7z', mode='r') as archive:
    archive.extractall(path='/content/data')


In [ ]:
# Image Preparation
from image_processor import ImageProcessor

csv_path = "/content/New_labels.csv"
image_folder = "/content/data/data/"

processor = ImageProcessor(csv_path, image_folder)
img_data, labels = processor.process_images()




In [ ]:
# Data Preparation
from data_preprocessor import DataPreprocessor
preprocessor = DataPreprocessor(img_data, labels)
train_images, val_images, train_labels, val_labels = preprocessor.preprocess_data()


In [ ]:
# Model Training
from speaker_agent import SpeakerAgent
agent = SpeakerAgent(train_images, train_labels, val_images, val_labels)
model = agent.create_model()


Epoch 1/30
400/400 [==============================] - 39s 58ms/step - loss: 3.9299 - accuracy: 0.0975 - val_loss: 8.0617 - val_accuracy: 0.0069
Epoch 2/30
400/400 [==============================] - 23s 58ms/step - loss: 2.3778 - accuracy: 0.2907 - val_loss: 2.0997 - val_accuracy: 0.3950
Epoch 3/30
400/400 [==============================] - 23s 58ms/step - loss: 1.7652 - accuracy: 0.4187 - val_loss: 1.2319 - val_accuracy: 0.6241
Epoch 4/30
400/400 [==============================] - 24s 59ms/step - loss: 1.4818 - accuracy: 0.5007 - val_loss: 0.9193 - val_accuracy: 0.7025
Epoch 5/30
400/400 [==============================] - 23s 58ms/step - loss: 1.2669 - accuracy: 0.5683 - val_loss: 1.6796 - val_accuracy: 0.5278
Epoch 6/30
400/400 [==============================] - 24s 59ms/step - loss: 1.1098 - accuracy: 0.6247 - val_loss: 0.7508 - val_accuracy: 0.7628
Epoch 7/30
400/400 [==============================] - 22s 55ms/step - loss: 0.9915 - accuracy: 0.6584 - val_loss: 0.5491 - val_accuracy:

In [ ]:
# Define the path to save the model
from tensorflow.keras.models import load_model
model_path = 'speaker_agent.h5'

model.save(model_path)
loaded_model = load_model(model_path)

# Game Round 1

In [ ]:
# Usage
from sklearn.preprocessing import LabelEncoder
import numpy as np
import pandas as pd
from image_predictor import ImagePredictor
# Load the Excel file into a pandas DataFrame
data = pd.read_csv("/content/New_labels.csv")

# Extract the filenames and labels into separate arrays
filenames = data['Filenames'].values
shapes = data['labels'].values
shapes = np.array(shapes)
label_encoder = LabelEncoder()
labels = label_encoder.fit_transform(shapes)
file_path = "/content/test_image.xlsx"
test_img_dir = "/content/test/"


test_data = pd.read_excel(file_path)
test_img = test_data['Filepath']
target_label = test_data['shapes']


# Create an instance of ImagePredictor with appropriate arguments
image_predictor = ImagePredictor(test_img_dir, loaded_model, label_encoder)

# Call the predict_labels method to get the predicted labels
predicted_labels = image_predictor.predict_labels(test_img, target_label)


1/1 [==============================] - 0s 22ms/step


In [ ]:
from listener_agent import ListenerAgent

file_path = "/content/test_image.xlsx"
test_data = pd.read_excel(file_path)
test_img = test_data['Filepath']
target_label = test_data['shapes']
test_img_dir = "/content/test/"
data_path = "/content/data/data/"


In [ ]:
# Usage example:
from Game_env_listerner_model2 import GameEnvironmentListerModel
listener_agent = ListenerAgent(data_path, filenames, shapes)
game_env_lister_model = GameEnvironmentListerModel(test_img_dir, test_img, predicted_labels, listener_agent, data_path, filenames, shapes)


In [ ]:
lis_image, lis_pred = game_env_lister_model.get_listener_model_predictions()

# Round 1 reward Accuracy

In [ ]:
from CollaborativeReward import CollaborativeReward
collab_reward_calculator = CollaborativeReward(predicted_labels,target_label, lis_pred)
collab_reward, true_conditions, false_conditions = collab_reward_calculator.calculate_reward()


[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


In [2]:
print(collab_reward)
collab_reward_percent = collab_reward/len(test_img)
print(collab_reward_percent)

6
0.075


# Game round 2

In [ ]:
from policy_updater import ModelUpdater
import tensorflow as tf

# Create an instance of the ModelUpdater class
model_updater = ModelUpdater(weights_path='speaker_agent.h5')

# Call the update_model method
model, lr = model_updater.update_model(loaded_model, collab_reward_percent, train_images, train_labels, val_images, val_labels, max_epochs=30)

# Save the updated model
model_path = 'speaker_agent_updated_1.h5'
model.save(model_path)

# Load the updated model
loaded_model = tf.keras.models.load_model(model_path)


Epoch 1/30
400/400 [==============================] - 28s 60ms/step - loss: 0.2671 - accuracy: 0.9113 - val_loss: 0.0441 - val_accuracy: 0.9925
Epoch 2/30
400/400 [==============================] - 24s 59ms/step - loss: 0.2490 - accuracy: 0.9178 - val_loss: 0.0506 - val_accuracy: 0.9906
Epoch 3/30
400/400 [==============================] - 24s 59ms/step - loss: 0.2243 - accuracy: 0.9272 - val_loss: 0.0447 - val_accuracy: 0.9919
Epoch 4/30
400/400 [==============================] - 21s 53ms/step - loss: 0.2194 - accuracy: 0.9287 - val_loss: 0.0552 - val_accuracy: 0.9897
Epoch 5/30
400/400 [==============================] - 23s 57ms/step - loss: 0.2164 - accuracy: 0.9284 - val_loss: 0.0614 - val_accuracy: 0.9859
Epoch 6/30
400/400 [==============================] - 23s 56ms/step - loss: 0.2066 - accuracy: 0.9298 - val_loss: 0.0319 - val_accuracy: 0.9950
Epoch 7/30
400/400 [==============================] - 23s 58ms/step - loss: 0.2065 - accuracy: 0.9326 - val_loss: 0.0388 - val_accuracy:

In [ ]:
# Create an instance of ImagePredictor with appropriate arguments
image_predictor = ImagePredictor(test_img_dir, loaded_model, label_encoder)

# Call the predict_labels method to get the predicted labels
predicted_labels = image_predictor.predict_labels(test_img, target_label)
listener_agent = ListenerAgent(data_path, filenames, shapes)
game_env_lister_model = GameEnvironmentListerModel(test_img_dir, test_img, predicted_labels, listener_agent, data_path, filenames, shapes)
lis_image, lis_pred = game_env_lister_model.get_listener_model_predictions()

1/1 [==============================] - 0s 30ms/step


In [ ]:
collab_reward_calculator = CollaborativeReward(predicted_labels,target_label, lis_pred)
collab_reward, true_conditions, false_conditions = collab_reward_calculator.calculate_reward()


In [4]:
print(collab_reward)
collab_reward_percent = collab_reward/len(test_img)
print(collab_reward_percent)

9
0.1125


# Game Round 3

In [ ]:
# Create an instance of the ModelUpdater class
model_updater = ModelUpdater(weights_path='speaker_agent_updated_1.h5')

# Call the update_model method
model, lr = model_updater.update_model(loaded_model, collab_reward_percent, train_images, train_labels, val_images, val_labels, max_epochs=30)

# Save the updated model
model_path = 'speaker_agent_updated_2.h5'
model.save(model_path)

# Load the updated model
loaded_model = tf.keras.models.load_model(model_path)


Epoch 1/30
400/400 [==============================] - 29s 62ms/step - loss: 0.1445 - accuracy: 0.9534 - val_loss: 0.0344 - val_accuracy: 0.9947
Epoch 2/30
400/400 [==============================] - 26s 65ms/step - loss: 0.1448 - accuracy: 0.9552 - val_loss: 0.0629 - val_accuracy: 0.9881
Epoch 3/30
400/400 [==============================] - 24s 60ms/step - loss: 0.1401 - accuracy: 0.9545 - val_loss: 0.0289 - val_accuracy: 0.9947
Epoch 4/30
400/400 [==============================] - 23s 57ms/step - loss: 0.1352 - accuracy: 0.9566 - val_loss: 0.0376 - val_accuracy: 0.9934
Epoch 5/30
400/400 [==============================] - 23s 58ms/step - loss: 0.1253 - accuracy: 0.9588 - val_loss: 0.0434 - val_accuracy: 0.9912
Epoch 6/30
400/400 [==============================] - 24s 59ms/step - loss: 0.1286 - accuracy: 0.9572 - val_loss: 0.0364 - val_accuracy: 0.9941
Epoch 7/30
400/400 [==============================] - 25s 61ms/step - loss: 0.1216 - accuracy: 0.9613 - val_loss: 0.0376 - val_accuracy:

In [ ]:
# Create an instance of ImagePredictor with appropriate arguments
image_predictor = ImagePredictor(test_img_dir, loaded_model, label_encoder)

# Call the predict_labels method to get the predicted labels
predicted_labels = image_predictor.predict_labels(test_img, target_label)
listener_agent = ListenerAgent(data_path, filenames, shapes)
game_env_lister_model = GameEnvironmentListerModel(test_img_dir, test_img, predicted_labels, listener_agent, data_path, filenames, shapes)
lis_image, lis_pred = game_env_lister_model.get_listener_model_predictions()

1/1 [==============================] - 0s 21ms/step


In [ ]:
collab_reward_calculator = CollaborativeReward(predicted_labels,target_label, lis_pred)
collab_reward, true_conditions, false_conditions = collab_reward_calculator.calculate_reward()


In [6]:
print(collab_reward)
collab_reward_percent = collab_reward/len(test_img)
print(collab_reward_percent)

11
0.1375
